In [11]:
from MyCreds.mycreds import USCensusAPI
import requests
import pandas as pd
import ast
from tqdm.notebook import tqdm

In [2]:
def get_variable_table_df(year):
    variable_table_url = f'https://api.census.gov/data/{year}/acs/acs1/variables.html'
    v_table = pd.read_html(variable_table_url)
    variable_df = pd.DataFrame(v_table[0])

    return variable_df

In [3]:
def get_male_by_age_index(variable_table):
    start_index = variable_table[((variable_table['Label'] == 'Estimate!!Total!!Male') | (variable_table['Label'] == 'Estimate!!Total:!!Male:')) & (variable_table['Concept'] == 'SEX BY AGE')].index[0]
    end_index = variable_table[((variable_table['Label'] == 'Estimate!!Total!!Male!!85 years and over') | (variable_table['Label'] == 'Estimate!!Total:!!Male:!!85 years and over')) & (variable_table['Concept'] == 'SEX BY AGE')].index[0]
    return start_index, end_index + 1

In [4]:
def get_variable_names(variable_table, indeces):
    total_male_by_age_variables = ",".join(variable_table.iloc[indeces[0]: indeces[1]]['Name'].values)
    return total_male_by_age_variables

In [5]:
def get_query_url(year, variables):
    # API Reference: https://www.census.gov/data/developers/guidance/api-user-guide.Example_API_Queries.html
    host = 'https://api.census.gov/data'
    year = f'/{year}'
    # Data Dictionary: https://api.census.gov/data.html
    dataset_acronym = '/acs/acs1'
    g = '?get='
    # Variables for the ACS: https://api.census.gov/data/2005/acs/acs1/variables.html
    # variables = 'NAME,B01001_001E'
    location = '&for=us:*'
    usr_key = f"&key={USCensusAPI.api_key}"

    query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"

    return query_url

In [6]:
def get_query_text(query_url):
    response = requests.get(query_url)
    return response.text

In [7]:
def get_values_from_response(response_text):
    values = [int(i) for i in ast.literal_eval(response_text)[1][:-1]]
    return values

In [8]:
def get_labels(variable_df, indeces):
    # labels = ['Male: Total', *[i.strip('Estimate!!Total:!!').replace("!!", " ") for i in variable_df.iloc[indeces[0]:indeces[1]]['Label'].values]]
    labels = [i.replace("!!", " ").replace(":", "") for i in variable_df.iloc[indeces[0]:indeces[1]]['Label'].values]
    return labels

In [9]:
def create_year_pop_dataframe(year, labels, values):
    df = pd.DataFrame({year: {labels[i]: values[i] for i in range(len(labels))}}).reindex(labels)
    return df

In [10]:
def create_male_pop_by_age_df(year):
    v_table = get_variable_table_df(year)
    male_by_age_indeces = get_male_by_age_index(v_table)
    variables = get_variable_names(v_table, male_by_age_indeces)
    query_url = get_query_url(year, variables)
    response_text = get_query_text(query_url)
    vals = get_values_from_response(response_text)
    labels = get_labels(v_table, male_by_age_indeces)
    df = create_year_pop_dataframe(year, labels, vals)
    return df

In [41]:
years = [i for i in range(2005, 2020)]
male_pop_by_age_df = pd.DataFrame(columns=['Population Label'])
for year in tqdm(years):
    try:
        y_df = create_male_pop_by_age_df(year).reset_index().rename({'index': 'Population Label'}, axis=1)
        male_pop_by_age_df = pd.merge(male_pop_by_age_df, y_df, how='outer', on='Population Label')
    except IndexError:
        next

  0%|          | 0/15 [00:00<?, ?it/s]

In [42]:
male_pop_by_age_df

,Population Label,2005,2006,2007,2008,2009,2012,2013,2014,2015,2016,2017,2018,2019
0,Estimate Total Male,141274964,147434940,148639222,149863485,151375321,154436243,155627698,156890101,158167834,159061631,160402504,161118151,161588973
1,Estimate Total Male Under 5 years,10356773,10421888,10578358,10707049,10853263,10166500,10109150,10117103,10112826,10130841,10138447,10047966,9938937
2,Estimate Total Male 5 to 9 years,9945226,10098366,10087957,10154505,10273948,10493216,10516217,10511796,10507658,10487494,10223398,10095699,10033518
3,Estimate Total Male 10 to 14 years,10674272,10587454,10513768,10428924,10532166,10601990,10622312,10596279,10533944,10579748,10854851,11000943,10987313
4,Estimate Total Male 15 to 17 years,6415594,6641247,6679175,6563046,6487137,6432763,6402435,6390741,6470980,6480400,6456448,6394674,6361859
5,Estimate Total Male 18 and 19 years,3642423,4504703,4573863,4678168,4696946,4613289,4556876,4550073,4495739,4511323,4547414,4578048,4541794
6,Estimate Total Male 20 years,1946123,2346861,2358876,2343397,2381015,2463952,2450726,2416850,2378536,2355103,2354931,2334371,2318283
7,Estimate Total Male 21 years,1926099,2268506,2269700,2239549,2285493,2444471,2398385,2375223,2357032,2318893,2284991,2261277,2257008
8,Estimate Total Male 22 to 24 years,5861680,6205974,6187367,6228461,6383204,6590360,6808517,6830222,6833725,6731345,6640221,6526051,6439169
9,Estimate Total Male 25 to 29 years,9657845,10356272,10505697,10713487,11006461,10760625,10867688,11070617,11290711,11496946,11720804,11869477,11817829


In [24]:
df_2018 = create_male_pop_by_age_df(2018).reset_index().rename({'index': 'Population Label'}, axis=1)
df_2018

,Population Label,2018
0,Estimate Total Male,161118151
1,Estimate Total Male Under 5 years,10047966
2,Estimate Total Male 5 to 9 years,10095699
3,Estimate Total Male 10 to 14 years,11000943
4,Estimate Total Male 15 to 17 years,6394674
5,Estimate Total Male 18 and 19 years,4578048
6,Estimate Total Male 20 years,2334371
7,Estimate Total Male 21 years,2261277
8,Estimate Total Male 22 to 24 years,6526051
9,Estimate Total Male 25 to 29 years,11869477


In [22]:
df_2019 = create_male_pop_by_age_df(2019).reset_index().rename({'index': 'Population Label'}, axis=1)
df_2019

,Population Label,2019
0,Estimate Total Male,161588973
1,Estimate Total Male Under 5 years,9938937
2,Estimate Total Male 5 to 9 years,10033518
3,Estimate Total Male 10 to 14 years,10987313
4,Estimate Total Male 15 to 17 years,6361859
5,Estimate Total Male 18 and 19 years,4541794
6,Estimate Total Male 20 years,2318283
7,Estimate Total Male 21 years,2257008
8,Estimate Total Male 22 to 24 years,6439169
9,Estimate Total Male 25 to 29 years,11817829


In [25]:
df_2018.merge(df_2019, on='Population Label')

,Population Label,2018,2019
0,Estimate Total Male,161118151,161588973
1,Estimate Total Male Under 5 years,10047966,9938937
2,Estimate Total Male 5 to 9 years,10095699,10033518
3,Estimate Total Male 10 to 14 years,11000943,10987313
4,Estimate Total Male 15 to 17 years,6394674,6361859
5,Estimate Total Male 18 and 19 years,4578048,4541794
6,Estimate Total Male 20 years,2334371,2318283
7,Estimate Total Male 21 years,2261277,2257008
8,Estimate Total Male 22 to 24 years,6526051,6439169
9,Estimate Total Male 25 to 29 years,11869477,11817829


In [39]:
df = pd.DataFrame(columns=['Population Label'])
df = pd.merge(df, df_2019, how='outer', on='Population Label')
df

,Population Label,2019
0,Estimate Total Male,161588973
1,Estimate Total Male Under 5 years,9938937
2,Estimate Total Male 5 to 9 years,10033518
3,Estimate Total Male 10 to 14 years,10987313
4,Estimate Total Male 15 to 17 years,6361859
5,Estimate Total Male 18 and 19 years,4541794
6,Estimate Total Male 20 years,2318283
7,Estimate Total Male 21 years,2257008
8,Estimate Total Male 22 to 24 years,6439169
9,Estimate Total Male 25 to 29 years,11817829


In [40]:
df = pd.merge(df, df_2018, how='outer', on='Population Label')
df

,Population Label,2019,2018
0,Estimate Total Male,161588973,161118151
1,Estimate Total Male Under 5 years,9938937,10047966
2,Estimate Total Male 5 to 9 years,10033518,10095699
3,Estimate Total Male 10 to 14 years,10987313,11000943
4,Estimate Total Male 15 to 17 years,6361859,6394674
5,Estimate Total Male 18 and 19 years,4541794,4578048
6,Estimate Total Male 20 years,2318283,2334371
7,Estimate Total Male 21 years,2257008,2261277
8,Estimate Total Male 22 to 24 years,6439169,6526051
9,Estimate Total Male 25 to 29 years,11817829,11869477
